In [15]:
import pandas as pd
from faker import Faker
import random

# Inicializando o Faker e definindo a localização para Brasil
fake = Faker('pt_BR')

# Lista de cargos
cargos = [
    "Não",
    "Celebrações",
    "Técnica",
    "MIDAF - Dança",
    "Infantil",
    "Eventos/Cerimonial",
    "MILAF - Louvor",
    "Comunicação",
    "Intercessão",
    "Introdutores",
    "MJF - Jovens"
]

# Lista de sexos
sexos = ["Masculino", "Feminino"]

# Lista de estados civis
estados_civis = ["Solteiro(a)", "Casado(a)", "Divorciado(a)", "Viúvo(a)"]

# Lista de escolaridades
escolaridades = ["Fundamental", "Médio", "Superior", "Pós-graduação", "Mestrado", "Doutorado"]

# Lista de CEP
df_ceps = pd.read_csv('ceps.csv', header=None)
ceps = df_ceps[0].tolist()



# Função para gerar um telefone no padrão '(92)999961674'
def gerar_telefone():
    return f'(92)9{random.randint(1000, 9999)}{random.randint(1000, 9999)}'

# Função para gerar endereço em Manaus
def gerar_endereco():
    return fake.street_name(), fake.bairro(), "Manaus", "AM", fake.building_number()

# Função para gerar "sim" ou "não" para receber notícias
def receber_noticias():
    return random.choice(["Sim", "Não"])

# Gerando dados
data = []
num_registros = 100  # Ajuste o número de registros conforme necessário

for i in range(num_registros):
    rua, bairro, cidade, uf, numero = gerar_endereco()
    registro = {
        'cpf':                      fake.cpf(),
        'nome_completo':            fake.name(),
        'data_nascimento':          fake.date_of_birth(minimum_age=18, maximum_age=80).strftime("%Y-%m-%d"),
        'genero':                   random.choice(sexos),
        'numero_celular':           gerar_telefone(),
        'email':                    fake.email(),
        'estado_civil':             random.choice(estados_civis),
        'numero_celula':            random.randint(1, 10),
        'participacao_ministerio':  random.choice(cargos),
        'data_batismo':             fake.date_this_decade().strftime("%Y-%m-%d"),
        'data_conversao':           fake.date_this_year().strftime("%Y-%m-%d"),
        'escolaridade':             random.choice(escolaridades),
        'profissao':                fake.job(),
        'cep':                      random.choice(ceps),
        'rua':                      rua,
        'bairro':                   bairro,
        'cidade':                   cidade,
        'uf':                       uf,
        'numero':                   random.randint(1, 999),
        'receber_noticias':         receber_noticias(),
        'id':                       i + 1
    }
    data.append(registro)

# Criando DataFrame
df = pd.DataFrame(data)

# Salvando em um arquivo CSV
df.to_csv('dados_falsos.csv', index=False)

print("Dados falsos gerados e salvos em 'dados_falsos.csv'.")


Dados falsos gerados e salvos em 'dados_falsos.csv'.


In [16]:
dados_falsos = pd.read_csv("dados_falsos.csv")
dados_falsos.head()

,cpf,nome_completo,data_nascimento,genero,numero_celular,email,estado_civil,numero_celula,participacao_ministerio,data_batismo,...,escolaridade,profissao,cep,rua,bairro,cidade,uf,numero,receber_noticias,id
0,796.310.845-66,Ana Liz Pacheco,1944-03-09,Masculino,(92)924786259,thalescorreia@example.com,Divorciado(a),6,Técnica,2022-01-21,...,Pós-graduação,Mensageiro,69065-152,Ladeira Machado,Vila Jardim Leblon,Manaus,AM,88,Não,1
1,416.375.289-73,Sophie Novais,1987-02-07,Feminino,(92)921105345,benjamin39@example.org,Viúvo(a),6,Intercessão,2022-04-07,...,Pós-graduação,Tatuador,69029-460,Viaduto Ester Rezende,Vila Independencia 2ª Seção,Manaus,AM,255,Sim,2
2,765.810.234-26,Augusto da Cunha,1958-10-23,Masculino,(92)997128238,eloa85@example.net,Divorciado(a),6,Intercessão,2024-06-09,...,Doutorado,Flanelinha,69065-125,Fazenda Samuel Carvalho,Lagoinha Leblon,Manaus,AM,491,Sim,3
3,863.912.045-51,Srta. Amanda Rocha,1959-10-07,Feminino,(92)994302191,zcasa-grande@example.org,Viúvo(a),7,MIDAF - Dança,2020-11-15,...,Mestrado,Treinador,69029-060,Alameda Silva,Santa Cruz,Manaus,AM,295,Sim,4
4,859.246.130-89,Sra. Rebeca Fernandes,2006-04-06,Masculino,(92)951347946,isabelnogueira@example.net,Divorciado(a),1,Comunicação,2023-12-15,...,Superior,Médico geneticista,69065-224,Jardim Vasconcelos,Vila Nova Cachoeirinha 2ª Seção,Manaus,AM,771,Não,5


In [17]:
import pandas as pd
import mariadb

# Função para criar a conexão com o banco de dados
def create_connection():
    try:
        conexao = mariadb.connect(
            host="54.196.208.38",
            user="root",
            password="qiykGUao3R.D",
            database="IDPB",
            port=3306
        )
        return conexao
    except mariadb.Error as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

# Função para fechar a conexão com o banco de dados
def close_connection(conexao):
    if conexao:
        conexao.close()
        print("Conexão com o banco de dados foi encerrada.")

# Carregar os dados do CSV
df = pd.read_csv('dados_falsos.csv')

# Criar a conexão com o banco de dados
conexao = create_connection()

if conexao:
    cursor = conexao.cursor()
    
    try:
        # Dropar todos os dados da tabela membros
        cursor.execute("DELETE FROM membros")
        conexao.commit()
        print("Todos os dados da tabela membros foram deletados.")
        
        # Preparar a query de inserção
        insert_query = """
        INSERT INTO membros (
            cpf, 
            nome_completo, 
            data_nascimento, 
            genero, 
            numero_celular, 
            email,
            estado_civil, 
            numero_celula, 
            participacao_ministerio, 
            data_batismo,
            data_conversao, 
            escolaridade, 
            profissao, 
            cep, 
            rua, 
            bairro,
            cidade, 
            uf,
            numero,
            receber_noticias
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        """
        
        # Iterar sobre as linhas do DataFrame e inserir os dados na tabela
        for index, row in df.iterrows():
            cursor.execute(insert_query, (
                row['cpf'],
                row['nome_completo'],
                row['data_nascimento'],
                row['genero'],
                row['numero_celular'],
                row['email'],
                row['estado_civil'],
                row['numero_celula'],
                row['participacao_ministerio'],
                row['data_batismo'],
                row['data_conversao'],
                row['escolaridade'],
                row['profissao'],
                row['cep'],
                row['rua'],
                row['bairro'],
                row['cidade'],
                row['uf'],
                row['numero'],
                row['receber_noticias']
            ))
        
        # Confirmar as alterações
        conexao.commit()
        print("Dados inseridos na tabela membros com sucesso.")
        
    except mariadb.Error as e:
        print(f"Erro ao executar operações com o banco de dados: {e}")
    
    # Fechar a conexão
    close_connection(conexao)
else:
    print("Falha ao conectar ao banco de dados.")


Todos os dados da tabela membros foram deletados.
Dados inseridos na tabela membros com sucesso.
Conexão com o banco de dados foi encerrada.
